# Chapter 17 - Future를 이용한 동시성

긴 지연 시간 동안 CPU 클록을 낭비하지 않기 위해 네트워크 입출력을 효율적으로 처리하려면 동시성을 이용해야 한다.<br>
다음은 웹에서 20개의 국기 이미지를 순차적으로 내려받는 코드이다.

In [2]:
import os
import time
import sys
import requests

POP20_CC = 'CN IN US ID BR PK NG BD RU JP MX PH VN ET EG DE IR TR CD FR'.split()
BASE_URL = 'http://flupy.org/data/flags'
DEST_DIR = 'downloads/'

def save_flag(img, filename):
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)
    
def get_flag(cc):
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content

def show(text):
    print(text, end=' ')
    sys.stdout.flush()
    
def download_many(cc_list):
    for cc in sorted(cc_list):
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')
    return len(cc_list)

def main(download_many):
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloads in {:.2f}s'
    print(msg.format(count, elapsed))
    
if __name__ == '__main__':
    main(download_many)


BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloads in 5.43s


이를 concurrent.futures를 이용해서 다시 구현하면 다음과 같다.

In [3]:
from concurrent import futures

MAX_WORKERS = 20

def download_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + '.gif')
    return cc

def download_many2(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor:
        res = executor.map(download_one, sorted(cc_list))
    
    return len(list(res))

if __name__ == '__main__':
    main(download_many2)

PH RU EG MX JP CD ET US FR IN NG ID VN DE PK IR CN TR BR BD 
20 flags downloads in 1.28s


파이썬 3.4 표준 라이브러리에서 Future라는 이름을 가진 클래스는 concurrent.futures.Future와 asyncio.Future이다.<br>
이 두 Future 클래스의 객체는 완료되었을 수도 있고 아닐 수도 있는 지연된 계산을 표현하기 위해 사용된다.<br>
<br>
클라이언트 코드는 Future 객체의 상태를 직접 변경하면 안 된다.<br>
Future 객체가 나타내는 연산이 완료되었을 때, 동시성 프레임워크가 Future 객체의 상태를 변경하기 때문이다.<br>
우리는 이 객체의 상태가 언제 바뀔지 제어할 수 없다.

## 블로킹 I/O와 GIL

CPython 인터프리터는 내부적으로 스레드 안전하지 않으므로, 전역 인터프리터 락(GIL)을 가지고 있다. GIL은 한 번에 한 스레드만 파이썬 바이트코드를 실행하도록 제한한다. 그렇기 때문에 단일 파이썬 프로세스가 동시에 다중 CPU 코어를 사용할 수 없다.<br>
파이썬 코드를 작성할 때 우리는 GIL을 제어할 수 없지만, 내장 함수나 C로 작성된 확장은 시간이 오래 걸리는 작업을 실행할 때 GIL을 해제할 수 있다. 사실 C로 작성된 파이썬 라이브러리라는 GIL을 관리하고, 자신의 OS 스레드를 생성해서 가용한 CPU 코드를 모두 사용할 수 있다. 하지만 라이브러리 코드가 상당히 복잡해지므로 대부분의 라이브러리 제작자는 이런 방식으로 구현하지 않는다.<br>
그런데 블로킹 입출력을 실행하는 모든 표준 라이브러리 함수는 OS에서 결과를 기다리는 동안 GIL을 해제한다. 즉, 입출력 위주의 작업을 실행하는 파이썬 프로그램은 파이썬으로 구현하더라도 스레드를 이용함으로써 이득을 볼 수 있다는 것이다. 파이썬 스레드가 네트워크로부터의 응답을 기다리는 동안, 블로킹된 입출력 함수가 GIL을 해제함으로써 다른 스레드가 실행될 수 있다.